# NOTEBOOK 1: OBTENCIÓN DE LOS DATOS
Trabajo fin de bootcamp

**Night Traders:**

<font color = "blue">Illari Alvarez-Gil, Andrés Guzzo, Dairon Andrés Benites, José Carlos Amo</font>

Predicción del cierre de huecos nocturnos bursátiles mediante el análisis de índices y variables financieras

El objetivo de este proyecto es la predicción del cierre de huecos nocturnos bursátiles bajista en un dia. Para ello, se trabajaron utilizando diversos índices bursátiles y variables financieras. Comenzamos utilizando datos de Alpha Advantage, pero debido a complicaciones en la obtención de datos, decidimos descargarlos de Yahoo Finance.

Desarrollo: Una vez que obtuvimos los datos de Yahoo Finance, comenzamos a agregar índices bursátiles y otras variables que podrían ser relevantes para nuestro objetivo. Hemos creado funciones que realizan el calculo de los índices bursátiles y las variables financieras. Evaluamos varias combinaciones de índices y variables para determinar cuáles eran las más importantes y necesarias según nuestro objetivo.

Con el fin de mejorar nuestras predicciones, implementamos una función que desplaza todos los índices bursátiles un día hacia adelante. Este ajuste nos permite analizar cómo las variables en un día específico podrían afectar el cierre de huecos nocturnos del siguiente día.

En resumen, nuestro proyecto se enfoca en analizar y predecir el cierre de huecos nocturnos bursátiles utilizando diferentes índices bursátiles y variables financieras. Hemos logrado mejorar nuestras predicciones mediante el ajuste de los índices y la implementación de una función que desplaza los datos un día hacia adelante.**

In [ ]:
!pip install pytz
!pip install ta
#!pip install pandas_ta


import yfinance as yf
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from google.colab import files
import pytz
import pandas as pd
import numpy as np
import ta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29088 sha256=9d8f3a4acce26290539ca22ea9ab08aa28c1066e89aa2b7750927388bd47bb20
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


**Descarga las cotizaciones de los valores del IBEX35 desde el 3/1/2000 hasta el 25/4/2023**

In [ ]:

# Descarga las cotizaciones de los siguientes valores:
ibex35 = ["ANA.MC", "ACX.MC", "ACS.MC", "AENA.MC", "AMS.MC", "MTS.MC", "BBVA.MC", "SAB.MC", "SAN.MC", "BKT.MC", "CABK.MC", "CLNX.MC", "ANE.MC", "ENG.MC", "ELE.MC", "FER.MC", "FDR.MC", "GRF.MC", "IAG.MC", "IBE.MC", "ITX.MC", "IDR.MC", "COL.MC", "ROVI.MC", "LOG.MC", "MAP.MC", "MEL.MC", "MRL.MC", "NTGY.MC", "RED.MC", "REP.MC", "SCYR.MC", "SLR.MC", "TEF.MC", "UNI.MC"]

end_date = datetime(2023, 4, 28, tzinfo=pytz.timezone('Europe/Madrid'))                         # hasta esta fecha
start_date = end_date - timedelta(days=25*365)   #Ojo cambiar days=0.15*365 por days=20*365   # desde esta fecha

def get_stock_data(stock):
    try:
        ticker = yf.Ticker(stock)
        data = ticker.history(start=start_date, end=end_date)
        data["company"] = ticker.info["longName"]
        return data
    except:
        print(f"\033[91mError retrieving data for {stock}\033[0m")
        return None

with ThreadPoolExecutor() as executor:
    data = list(tqdm(executor.map(get_stock_data, ibex35), total=len(ibex35)))

df = pd.concat(data)
df.to_csv('ibex35_final.csv')
files.download('ibex35_final.csv')



  0%|          | 0/35 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Comprobamos que los datos están disponibles sin incidencias**

**Una vez descargados los datos, vemos que disponemos de los valores de Apertura, Máximo, Mínimo, Cierre, Volúmen de contratacion, si ha habido pago de Dividendos o Splits y el nombre de la empresa cotizada.
Como esas variables son insuficientes para desarrollar nuestroproyecto, añadimos nuevas features con indicadores técnicos bursátiles. Esos indicadores se calculan con la librería TA**

**Definimos cada una de las funciones que usaremos para crear las nuevas features basadas en indicadores técnicos bursátiles**

In [ ]:

# SMA MEDIA MOVIL SIMPLE 
'''
Algunos traders de gaps prefieren períodos de 5, 10 o 20 días para SMA, pues dan una indicación más sensible de los movimientos recientes del precio.
Es posible calcular el SMA utilizando diferentes tipos de datos, como el precio de cierre, precios máximos o mínimos, o volumen, porque estos
indicadores son calculados mediante fórmulas matemáticas que utilizan los datos del precio de los activos, independientemente de si son precios de 
apertura, cierre, máximos o mínimos
'''
def SMA(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        col_name = 'SMA_'+ str(periodo)  # nombre de la nueva columna que se va a crear
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            group[col_name + '_' + col] = ta.trend.ema_indicator(group[col], window=periodo)
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final

# EMA MEDIA MOVIL EXPONENCIAL 
'''
Algunos traders de gaps prefieren períodos de 5, 10 o 20 días para EMA, pues dan una indicación más sensible de los movimientos recientes del precio.
Es posible calcular el EMA utilizando diferentes tipos de datos, como el precio de cierre, precios máximos o mínimos, o volumen, porque estos
indicadores son calculados mediante fórmulas matemáticas que utilizan los datos del precio de los activos, independientemente de si son precios de 
apertura, cierre, máximos o mínimos 
'''
def EMA(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        col_name = 'EMA_'+ str(periodo)  # nombre de la nueva columna que se va a crear
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            group[col_name + '_' + col] = ta.trend.ema_indicator(group[col], window=periodo)
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final

# MACD Moving Average Convergence Divergence
'''
No hay valores específicos de window_slow, window_fast y window_sign en el MACD que se consideren óptimos para el gap trading, ya que dependerá del
marco temporal utilizado, así como de la volatilidad del activo que se esté operando.
Para trading normal a corto plazo, los valores típicos para el MACD son:
  window_fast: 12
  window_slow: 26
  window_sign: 9
Es posible calcular el MACD utilizando diferentes tipos de datos, como el precio de cierre, precios máximos o mínimos, o volumen, porque estos
indicadores son calculados mediante fórmulas matemáticas que utilizan los datos del precio de los activos, independientemente de si son precios de 
apertura, cierre, máximos o mínimos 
'''
def MACD(data, window_slow, window_fast, window_sign):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        col_name = 'MACD_'+ str(window_slow) + '_' + str(window_fast) + '_' + str(window_sign)  # nombre de la nueva columna que se va a crear
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            macd = ta.trend.MACD(group[col], window_slow=window_slow, window_fast=window_fast, window_sign=window_sign)
            group[col_name + '_' + col] = macd.macd()
            group[col_name + '_' + col + '_signal'] = macd.macd_signal()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final



# RSI Relative Strength Index
'''
El periodo que los inversores utilizan para determinar el RSI puede variar. Catorce días es un período común, pero algunos inversores 
experimentados utilizan períodos más cortos o largos. Un periodo más corto suele dar más señales que un periodo más largo.
El RSI debe ser calculado sobre los precios de cierre exclusivamente.
Compara la magnitud de las ganancias y pérdidas recientes durante un período de tiempo específico para medir la velocidad y el cambio 
de los movimientos de precios de un valor. Se utiliza principalmente para intentar identificar condiciones de sobrecompra o sobreventa en la 
negociación de un activo.

Suele calcularse sobre 14 días
''' 
def RSI(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        col_name = 'RSI_Close_'+ str(periodo)  # nombre de la nueva columna que se va a crear
        group['RSI'] = ta.momentum.RSIIndicator(group['Close'], window=periodo).rsi()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Bollinger Bands
'''
Los valores por defecto que se utilizan para su cálculo son de 21 para la media y 2 desviaciones estándar.
Si se reduce o se incrementa de forma significativa el valor de la media hay que ajustar en el mismo sentido el número de desviaciones estándar. 
Para valores de la media por encima de 50 (largo plazo) => 2,5 desviaciones; valores de la media cercanos a 10 (corto plazo)=> 1,5 desviaciones.4​2​

***************************************************
**********************IMPORTANTE!!!!!!!!!!!!!!!!!!!

El hband_indicator en las bandas de Bollinger es un indicador que toma valores booleanos (True o False) que indican si el precio 
del activo ha superado o no la banda superior de Bollinger.

El lband_indicator en las bandas de Bollinger también devuelve valores booleanos (True o False). En este caso, el lband_indicator indica si 
el precio del activo ha caído por debajo de la banda inferior de Bollinger.

'''
def BOLLINGER(data, periodo, dev):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        col_name = 'BOLLINGER_'+ str(periodo) + '_' + str(dev)  # nombre de la nueva columna que se va a crear
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            bb = ta.volatility.BollingerBands(group[col], window=periodo, window_dev=dev)
            group[col_name +'_high'] = bb.bollinger_hband()
            group[col_name +'_low'] = bb.bollinger_lband()
            group[col_name +'_medium'] = bb.bollinger_mavg()
            group[col_name +'_bbhi'] = bb.bollinger_hband_indicator()
            group[col_name +'_bbli'] = bb.bollinger_lband_indicator()
            group[col_name +'_bbw'] = bb.bollinger_wband()
            group[col_name +'_bbp'] = bb.bollinger_pband()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Añadiendo Desviación Estandar
def STD_DEV(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            col_name = 'STD_DEV_'+ str(periodo) +'_' + col # nombre de la nueva columna que se va a crear
            group[col_name] = group[col].rolling(window=periodo).std()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# CCI Commodity Channel Index.
'''
El índice del canal de productos básicos (CCI) es un oscilador introducido originalmente por Donald Lambert en 1980.
CCI mide la variación de un valor con respecto a la media estadística.
Para propósitos de escala, Lambert fijó la constante en 0.015 para asegurar que aproximadamente el 70 al 80 por ciento de los valores de 
CCI estarían entre −100 y +100. El CCI fluctúa por encima y por debajo de cero. El porcentaje de valores de CCI que se encuentran entre +100 y 
−100 dependerá de la cantidad de períodos utilizados. Un CCI más corto será más volátil con un porcentaje menor de valores entre +100 y −100. 
Por el contrario, cuantos más períodos se utilicen para calcular el CCI, mayor será el porcentaje de valores entre +100 y −100.

Lo habitual es calcularlo con periodo = 14  constante = 0.015
'''
def CCI(data, periodo, constant):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'CCI_Close_' + str(periodo) + '_' + str(constant) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group['CCI'] = ta.trend.CCIIndicator(group['High'], group['Low'], group['Close'], window=periodo, constant=constant).cci()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# ESTOCÁSTICO
'''
El Momento o Momentum es un indicador técnico que mide la aceleración y desaceleración en los cambios de los precios entre dos instantes en el 
tiempo, es decir, la diferencia entre el cierre de hoy y el cierre de hace “n” sesiones (generalmente 10 o 12 sesiones).
Los valores habituales son: periodo=14 , smooth_window=3
'''
def ESTOCASTICO(data, periodo, smooth):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'ESTOCASTICO_'+ str(periodo) + '_' + str(smooth) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        stoch = ta.momentum.StochasticOscillator(group['High'], group['Low'], group['Close'], window=periodo, smooth_window=smooth)
        group[col_name + '_%K'] = stoch.stoch()
        group[col_name + '_%D'] = stoch.stoch_signal()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# WILLIAMS R
'''
Este indicador es uno de los osciladores más populares entre aquellos que se utilizan para determinar si un mercado está sobrecomprado o sobrevendido. 
El indicador fue desarrollado por el famoso trader Larry Williams y es su herramienta técnica más conocida.
El indicador Williams refleja el nivel del precio de cierre en relación con el precio más elevado para el periodo de tiempo analizado. 

el indicador Williams R oscila en un rango de entre 0 y (-100), donde:
    Valores entre 0 y (-20) => Sobrecompra
    Valores entre (-80) y (-100) => Sobreventa

Lo habitual es calcularlo para un periodo de 14 días
'''
def WILLIAMSR(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'WILLIAMSR_'+ str(periodo) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group[col_name] = ta.momentum.WilliamsRIndicator(group['High'], group['Low'], group['Close'], lbp=periodo).williams_r()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Accumulation/Distribution Index (ADI)
'''
Desarrollado por Marc Chaikin, la línea de distribución de acumulación es un indicador basado en el volumen 
diseñado para medir el flujo acumulado de dinero que entra y sale de un valor.

La línea de distribución de acumulación es una medida acumulativa del flujo de volumen o flujo de dinero de cada período. 
Un multiplicador positivo alto combinado con un volumen alto muestra una fuerte presión de compra que empuja al indicador al alza. 
Por el contrario, un número negativo bajo combinado con un volumen alto refleja una fuerte presión de venta que empuja al indicador a la baja. 
El volumen de flujo de dinero se acumula para formar una línea que confirma o contradice la tendencia del precio subyacente. 
En este sentido, el indicador se utiliza para reforzar la tendencia subyacente o para poner en duda su sostenibilidad. 
Una tendencia alcista en los precios con una tendencia bajista en la línea de distribución de acumulación sugiere una presión de venta 
subyacente (distribución) que podría presagiar una reversión bajista en el gráfico de precios.
'''
def ADI(data):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        group['ADI'] = ta.volume.AccDistIndexIndicator(group['High'], group['Low'], group['Close'], group['Volume']).acc_dist_index()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Chaikin Money Flow (CMF)
'''
Desarrollado por Marc Chaikin, Chaikin Money Flow mide la cantidad de volumen de flujo de dinero durante un período específico. 
El Volumen de Flujo de Dinero forma la base para la Línea de Distribución de Acumulación. En lugar de un total acumulativo, 
Chaikin Money Flow suma el Volumen de flujo de dinero para un período retrospectivo específico, generalmente 20 o 21 días. 
El indicador resultante fluctúa por encima o por debajo de la línea cero como un oscilador.

Como se ha indicado el periodo habitual es de 20 o 21 días
'''
def CMF(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'CMF_'+ str(periodo) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group[col_name] = ta.volume.ChaikinMoneyFlowIndicator(group['High'], group['Low'], group['Close'], group['Volume'], window=periodo).chaikin_money_flow()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Parabolic SAR
'''
ElParabólicoStop and Reverse, más comúnmente conocido como Parabolic SAR, es un indicador de seguimiento de tendencias desarrollado por J. Welles Wilder. 
El SAR parabólico se muestra como una sola línea parabólica (o puntos) debajo de las barras de precios en una tendencia alcista y encima de las 
barras de precios en una tendencia bajista.

el Paso, también conocido como Factor de Aceleración (AF), es un multiplicador que influye en la tasa de cambio en SAR. Los usuarios de SharpCharts 
pueden establecer el Paso y el Paso máximo. El Paso aumenta gradualmente a medida que la tendencia se extiende hasta alcanzar el máximo establecido 
por el usuario. El Paso dicta la sensibilidad del indicador SAR.

La sensibilidad SAR se puede disminuir disminuyendo el Paso . Un paso más bajo mueve el SAR más lejos del precio, lo que hace que una reversión sea 
menos probable. De manera similar, la sensibilidad SAR se puede aumentar aumentando el paso. Un paso más alto acerca el SAR a la acción del precio, lo 
que hace que sea más probable una reversión. El indicador se invertirá con demasiada frecuencia si el paso se configura demasiado alto. 
Esto producirá señales falsas y no logrará capturar la tendencia.

step ( flotante ): el factor de aceleración utilizado para calcular el SAR. Su valor habitual es 0.02
max_step ( float ): el valor máximo permitido para el factor de aceleración. Su valor habitual es 0.2

***************************************************
**********************IMPORTANTE!!!!!!!!!!!!!!!!!!!
Tanto el psar_down_indicator() como el psar_up_indicator() son indicadores que devuelven valores booleanos (True o False).

El psar_down_indicator() indica si el precio del activo ha caído por debajo del valor del indicador de Parabolic SAR (PSAR) para la dirección bajista.

El psar_up_indicator() indica si el precio del activo ha superado el valor del indicador de Parabolic SAR (PSAR) para la dirección alcista.

'''
def PSAR(data, step, maxstep):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'PSAR_'+ str(step) + '_' +  str(maxstep) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        psar = ta.trend.PSARIndicator(group['High'], group['Low'], group['Close'], step = step , max_step = maxstep)
        group[col_name] = psar.psar() # valor PSAR
        group[col_name + '_down'] = psar.psar_down() # Valor de tendencia descendente de PSAR
        group[col_name + '_down_indicator'] = psar.psar_down_indicator() # Indicador de valor de tendencia descendente de PSAR
        group[col_name + '_up'] = psar.psar_up() # Valor de tendencia ascendente de PSAR
        group[col_name + '_up_indicator'] = psar.psar_up_indicator() # Indicador de valor de tendencia ascendente de PSAR
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# ATR (Average True Range)
'''
Desarrollado por J. Welles Wilder, el Average True Range (ATR) es un indicador que proporciona una indicación del grado de 
volatilidad de los precios. Los movimientos fuertes, en cualquier dirección, suelen ir acompañados de grandes rangos o grandes rangos verdaderos.

Wilder comenzó con un concepto llamado True Range (TR) , que se define como el mayor de los siguientes:

Método 1: Máximo actual menos el Mínimo actual
Método 2: máximo actual menos el cierre anterior (valor absoluto)
Método 3: Mínimo actual menos el cierre anterior (valor absoluto)
Los valores absolutos se utilizan para garantizar números positivos. Después de todo, Wilder estaba interesado en medir la distancia entre dos puntos, 
no la dirección. Si el máximo del período actual está por encima del máximo del período anterior y el mínimo está por debajo del mínimo del período 
anterior, entonces el rango de máximos y mínimos del período actual se utilizará como rango verdadero. Este es un "outside day" que usaría 
el Método 1 para calcular el TR. Esto es bastante sencillo. Los métodos 2 y 3 se utilizan cuando hay un hueco o un día interior. 
Se produce una brecha cuando el cierre anterior es mayor que el máximo actual (lo que indica una brecha potencial a la baja o un movimiento límite) 
o el cierre anterior es menor que el mínimo actual (lo que indica una brecha potencial al alza o un movimiento límite).

Suele calcularse para un periodo de 14 días
'''
def ATR(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'ATR_'+ str(periodo) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group[col_name] = ta.volatility.AverageTrueRange(group['High'], group['Low'], group['Close']).average_true_range()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# ADX (Average Directional Index)
'''
Índice de movimiento direccional promedio (ADX)

El indicador direccional positivo (+DI) y el indicador direccional negativo (-DI) se derivan de promedios suavizados de estas 
diferencias y miden la dirección de la tendencia a lo largo del tiempo. Estos dos indicadores a menudo se denominan colectivamente Indicador 
de movimiento direccional (DMI).

El índice direccional promedio (ADX) se deriva a su vez de los promedios suavizados de la diferencia entre +DI y -DI, ​​y mide la fuerza de 
la tendencia (independientemente de la dirección) a lo largo del tiempo.

Usando estos tres indicadores juntos, los traders pueden determinar tanto la dirección como la fuerza de la tendencia.

Suele calcularse para un periodo de 14 días
'''
def ADX(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'ADX_'+ str(periodo) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        adx = ta.trend.ADXIndicator(group['High'], group['Low'], group['Close'],  window = periodo)
        group[col_name] = adx.adx() # valor del Índice direccional promedio (ADX)
        group[col_name + '_neg'] = adx.adx_neg( ) # Valor de Indicador direccional negativo (-DI)
        group[col_name + '_pos'] = adx.adx_pos( ) # Valor de Indicador direccional positivo (+DI)
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# ROC (Rate of Change) También llamado MOMENTUM
'''
Tasa de cambio (ROC) o Ritmo de cambio, que también se conoce simplemente como Momentum, es un oscilador de impulso puro que mide el porcentaje 
de cambio en el precio de un período al siguiente. El cálculo de ROC compara el precio actual con el precio de hace "n" períodos. 
El gráfico forma un oscilador que fluctúa por encima y por debajo de la línea cero a medida que la tasa de cambio se mueve de positiva a negativa. 
Como oscilador de impulso, las señales de ROC incluyen cruces de línea central, divergencias y lecturas de sobrecompra y sobreventa. 
Las divergencias no logran presagiar reversiones la mayoría de las veces, por lo que este artículo prescindirá de una discusión detallada sobre ellas. 
Aunque los cruces de la línea central son propensos a latigazos, especialmente a corto plazo, estos cruces se pueden usar para identificar 
la tendencia general. Identificar los extremos de sobrecompra o sobreventa es algo natural para el oscilador de tasa de cambio.

El periodo de cálculo suele ser de 12 días
'''
def ROC(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            col_name = 'ROC_'+ col + '_' + str(periodo) # nombre de la nueva columna que se va a crear
            group[col_name] = ta.momentum.ROCIndicator(group[col]).roc()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# OBV (On Balance Volume)
'''
El volumen en balance ( OBV ) es un indicador de análisis técnico destinado a relacionar el precio y el volumen en el mercado de valores. 
El OBV se basa en un volumen total acumulativo.

Al volumen total de cada día se le asigna un valor positivo o negativo dependiendo de si los precios son más altos o más bajos ese día. 
Un cierre más alto da como resultado que el volumen de ese día obtenga un valor positivo, mientras que un cierre más bajo da como resultado un 
valor negativo. [2] Entonces, cuando los precios suben, el OBV también debería subir, y cuando los precios alcanzan un nuevo repunte alto, el OBV 
también debería subir. Si el OBV no logra superar su máximo anterior, entonces se trata de una divergencia negativa , lo que sugiere un movimiento débil.

La técnica, originalmente llamada "volumen continuo" por Woods y Vignola, luego fue nombrada "volumen en equilibrio" por Joseph Granville, 
quien popularizó la técnica en su libro de 1963 Granville's New Key to Stock Market Profits . [1] El índice se puede aplicar a acciones 
individualmente en función de su cierre diario al alza o a la baja, o al mercado en su conjunto, utilizando la amplitud de los datos del mercado, 
es decir, la relación avance/declive.

OBV se usa generalmente para confirmar movimientos de precios. [4] La idea es que el volumen es mayor en los días en que el movimiento del 
precio está en la dirección dominante, por ejemplo, en una fuerte tendencia alcista hay más volumen en los días alcistas que en los bajistas.
'''
def OBV(data):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        group['OBV'] = ta.volume.OnBalanceVolumeIndicator(group['Close'], group['Volume']).on_balance_volume()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


#  MFI Money Flow Index
'''
Utiliza tanto el precio como el volumen para medir la presión de compra y venta. Es positivo cuando el precio típico sube (presión de compra) 
y negativo cuando el precio típico baja (presión de venta). Luego, se conecta una proporción de flujo de dinero positivo y negativo en una 
fórmula RSI para crear un oscilador que se mueve entre cero y cien.

Se suele calcular para periodos de 14 días
'''
def MFI(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    col_name = 'MFI_'+ str(periodo) # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group[col_name] = ta.volume.MFIIndicator(group['High'], group['Low'], group['Close'], group['Volume'], window=periodo).money_flow_index()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# ICHIMOKU CLOUD
'''
Identifica la tendencia y busca señales potenciales dentro de esa tendencia.
Para valores típicos de p_bajo=9, p_medio=26 y p_alto=52
'''
def ICHIMOKU (data, p_bajo, p_medio, p_alto):
    data_grouped = data.groupby("company")
    datas = []
    col_name_a = 'ICHIMOKU_A_'+ str(p_bajo) + '_' + str(p_medio)  # nombre de la nueva columna que se va a crear
    col_name_b = 'ICHIMOKU_B_'+ str(p_medio) + '_' + str(p_alto)  # nombre de la nueva columna que se va a crear
    col_name_base = 'ICHIMOKU_BASE_LINE_'+ str(p_bajo) + '_' + str(p_medio)  # nombre de la nueva columna que se va a crear
    col_name_conversion = 'ICHIMOKU_CONVERSION_LINE_'+ str(p_bajo) + '_' + str(p_medio)  # nombre de la nueva columna que se va a crear
    for name, group in data_grouped:
        group[col_name_a] = ta.trend.ichimoku_a(group['High'], group['Low'], window1=p_bajo , window2=p_medio)
        group[col_name_b] = ta.trend.ichimoku_b(group['High'], group['Low'], window2=p_medio , window3=p_alto)
        group[col_name_base] = ta.trend.ichimoku_base_line(group['High'], group['Low'], window1=p_bajo , window2=p_medio)
        group[col_name_conversion] = ta.trend.ichimoku_conversion_line(group['High'], group['Low'], window1=p_bajo , window2=p_medio)
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# DPO Detrended Price Oscillator
'''
El oscilador de precios sin tendencia (DPO) es un indicador diseñado para eliminar la tendencia del precio y facilitar la identificación de ciclos.
DPO no se extiende hasta la última fecha porque se basa en un promedio móvil desplazado. Sin embargo, la alineación con la fecha más reciente no es 
un problema porque DPO no es un oscilador de momento. En su lugar, DPO se utiliza para identificar ciclos máximos y mínimos y estimar la duración del ciclo.

El oscilador de precios sin tendencia (DPO) mide la diferencia entre un precio pasado y un promedio móvil.
Tenga en cuenta que DPO en sí mismo está desplazado hacia la izquierda. El indicador oscila por encima o por debajo de cero a medida que los 
precios se mueven por encima o por debajo de la media móvil desplazada. 

Suele calcularse con un periodo = 20
'''
def DPO(data, periodo):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
            col_name = 'DPO_'+ col + '_' + str(periodo) # nombre de la nueva columna que se va a crear
            group[col_name] = ta.trend.DPOIndicator(group[col], window=periodo).dpo()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# NVI Negative Volume Index
'''
El índice de volumen negativo (NVI) es un indicador acumulativo que utiliza el cambio de volumen para decidir cuándo está activo el 
dinero inteligente. Paul Dysart desarrolló por primera vez este indicador en la década de 1930. El índice de volumen negativo de Dysart 
funciona bajo el supuesto de que el dinero inteligente está activo en los días en que el volumen disminuye y el dinero no tan inteligente está 
activo en los días en que aumenta el volumen.

NVI cuenta los cambios de precio cuando el volumen disminuye y descuenta los cambios de precio cuando aumenta el volumen. 
La suposición es que el dinero inteligente (informado) está en funcionamiento cuando el volumen disminuye y el dinero no tan inteligente (sin información) 
está en funcionamiento cuando el volumen aumenta. Tenga en cuenta que este indicador fue diseñado para índices de mercado amplios y volumen de intercambio. 
Se puede usar en acciones y ETF, pero NVI no siempre actúa como se esperaba. NVI producirá excelentes señales de divergencia alcista/bajista con 
algunas acciones, pero se verá totalmente desincronizado con otras acciones. Como con todos los indicadores, NVI no debe usarse solo. 

'''
def NVI(data):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        group['NVI'] = ta.volume.NegativeVolumeIndexIndicator(group['Close'], group['Volume']).negative_volume_index()
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final



**Creamos esas nuevas features basadas en indicadores técnicos bursátiles**

In [ ]:
# Crea las nuevas variables basadas en indicadores técnicos bursátiles

df = SMA(df, periodo = 10)
df = EMA(df, periodo = 10)
df = MACD(df, window_slow = 6, window_fast = 12, window_sign = 9)
df = RSI(df, periodo = 14)
df = BOLLINGER(df, periodo = 21, dev = 2)
df = STD_DEV(df, periodo = 14)
df = CCI(df, periodo = 14, constant = 0.015)
df = ESTOCASTICO(df, periodo = 14, smooth = 3)
df = WILLIAMSR(df, periodo = 14)
df = ADI(df)
df = CMF(df, periodo = 21)
df = PSAR(df, step = 0.02, maxstep = 0.2)
df = ATR(df, periodo = 14) 
df = ADX(df, periodo = 14)
df = ROC(df, periodo = 12)
df = OBV(df)
df = MFI(df, periodo = 14)
df = ICHIMOKU (df, p_bajo = 9, p_medio = 26, p_alto = 52)
df = DPO(df, periodo = 20)
df = NVI(df)


**Definimos otras funciones que usaremos para crear nuevas features independientes de indicadores técnicos pero útiles para nuestro futuro análisis.**

In [ ]:
# Pre_Open
'''
Crea una nueva columna "PRE_OPEN" que en cada fila contiene el precio de cierre de la sesion anterior
'''
def agregar_pre_open(data):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        group['PRE_OPEN'] = group['Close'].shift(1)
        group['PRE_OPEN'] = group['PRE_OPEN'].fillna(group['Open'])
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final


# Cuantía de los Gap en valor absoluto y en porcentaje
'''
Crea dos nuevas columnas "Gap_%" y "Gap_€" que contienen respectivamente el % de gap respecto de la sesion anterior y el valor aboluto de dicho gap
'''
def agregar_gaps(data):
    data_grouped = data.groupby("company")
    datas = []
    for name, group in data_grouped:
        group['GAP_%'] = ((group['Open'] - group['PRE_OPEN']) / group['PRE_OPEN']) * 100
        group['GAP_€'] = group['Open'] - group['PRE_OPEN'] 
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final

# Dias en cerrarse el gap y beneficio por accion
'''
Crea dos nuevas columnas 'CLOSURE' y 'PROFIT_%'. La primera contiene el valor 1 si el gap se cierra en el mismo día en cuyo caso la segunda 
contine el beneficio en porcentaje por accion si se compra en la apertura y se vende en el maximo del dia de cierre del gap.
Por otra parte si 'CLOSURE' es 0 (no se cierra en el mismo día) entonces 'PROFIT_%' toma el valor -999
'''
def days_to_closure_and_profit(data):
    data_grouped = data.groupby('company')
    datas = []
    for name, group in data_grouped:
        for i, row in group.iterrows():
            if row['GAP_€'] < 0 and row['High'] >= row['PRE_OPEN']:
                group.loc[i, 'CLOSURE'] = 1
                group.loc[i, 'PROFIT_%'] = ((100 * row['High']) / row['Open']) - 100
            else:
                group.loc[i, 'CLOSURE'] = 0
                group.loc[i, 'PROFIT_%'] = 8888888888
        datas.append(pd.DataFrame(group))
                
    data_final = pd.concat(datas)
    return data_final


# Datos relativos a las fechas
'''
Crea una nueva columna "Weekday" con el número del día de la semana (de lunes=0 a domingo=6) correspondiente a cada fecha en el índice del DataFrame,
una nueva columna "Week" que contendrá el número de la semana en el año y una nueva columna "Month" que contendrá el número del mes 
(1 para enero, 2 para febrero, etc.).
La idea de esta variable es ver si el dia de la semana, al semana o el mes en que se produce el gap tiene que ver cómo se cierra dicho gap
'''

def date_features(data):
    data.index = pd.to_datetime(data.index)
    data["WEEKDAY"] = data.index.weekday
    data["WEEK"] = pd.Index(data.index.isocalendar().week, dtype='Int64')
    data["MONTH"] = data.index.month
    return data

'''
Reindexa de manera que se descartan los índices existentes y se crea un nuevo índice de números enteros consecutivos a partir de cero.
'''
def reindexa(data):
    data = data.reset_index(drop=True)
    return data


**Creamos esas nuevas features basadas en indicadores técnicos bursátiles**

In [ ]:
# Crea las nuevas variables basadas en indicadores técnicos bursátiles

df = agregar_pre_open(df)
df = agregar_gaps(df)
df = date_features(df)
df.reset_index(drop=True, inplace=True)
df = days_to_closure_and_profit(df)



**En nuestro problema existe la limitación de los datos disponibles en tiempo real para hacer predicciones una vez que el mercado acaba de abrir. Esta es una limitación común en el trading algorítmico.**

**En el momento de la apertura los unicos datos de nuestro df conocidos son 'Open', 'company', 'PRE_OPEN', 'GAP_%', 'GAP_€', 'WEEKDAY', 'WEEK' y 'MONTH'. Con tan pocas variables conocidas ser capaces de hacer un modelo que logre predicciones precisas con información tan limitada parece imposible.**

**La solución por la que hemos optado es modelar cada gap utilizando los datos de las features del día anterior. Dado que el tamaño del gap es una medida de la diferencia entre el precio de cierre de la sesión anterior y el precio de apertura de la sesión actual, es razonable suponer que las características del mercado del día anterior podrían ser útiles para predecir el tamaño del gap el día posterior.**

**Esto requiere modelar cada gap utilizando los datos de las características del día anterior, por lo que debemos realizar un desplazamiento (shift) de todas las columnas, excepto de la columna que contenga la variable objetivo (label) y excepto las columnas que contengan variables conocidas en el instante de la apertura.**

**Es importante decir que estamos teniendo en cuenta que el desplazamiento de las características podría introducir ciertos sesgos en el modelo, ya que la información más reciente y relevante para el mercado puede no estar reflejada en las características desplazadas.** 

In [ ]:
# Función para mover indicadores un día adelante
# excepto las variables ['Open', 'company', 'PRE_OPEN', 'GAP_%', 'GAP_€', 'WEEKDAY', 'WEEK', 'MONTH', 'CLOSURE', 'PROFIT_%']
def shift_rows(data):
    data_grouped = data.groupby("company")
    datas = []
    cols = data.columns
    cols_to_shift = [col for col in cols if col not in ['Open', 'company', 'PRE_OPEN', 'GAP_%', 'GAP_€', 'WEEKDAY', 'WEEK', 'MONTH', 'CLOSURE', 'PROFIT_%']]
    for name, group in data_grouped:
        group[cols_to_shift] = group[cols_to_shift].shift(1)
        datas.append(group)
    data_final = pd.concat(datas)
    return data_final

df = shift_rows(df)

**Vamos a pasar 'CLOSURE' a la primera posición en la tabla porque es nuestra label**

In [ ]:
# Pasar 'CLOSURE' a la primera posición en la tabla porque es nuestra label 
cols = df.columns.tolist()                        # Obtener los nombres de las columnas como una lista
cols.insert(0, cols.pop(cols.index('CLOSURE')))   # Eliminar la columna CLOSURE de su posición original y agregarla en la posición 0
df = df.reindex(columns=cols)                     # Reorganizar el DataFrame según la nueva lista de columnas

**Y como solo queremos modelar los gaps negativos, eliminamos todas las filas que tengan gaps iguales o mayores que 0**

In [ ]:
# Selecciona todas las filas en las que el valor de "GAP_€" es negativo
df = df.loc[df['GAP_€'] < 0]

**Guardamos el df con todas estas variables ampliadas a un archivo csv.**

In [ ]:
# Guardar datos con caracteristicas ampliadas en un nuevo archivo CSV
df.to_csv('expanded_features_final.csv')
files.download('expanded_features_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **APLICAMOS EL MISMO PROCESO A UN DF ACTUAL QUE ES EL DF OBJETIVO(CON DATOS ACTUALES)**

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait
ibex35 = ["ANA.MC", "ACX.MC", "ACS.MC", "AENA.MC", "AMS.MC", "MTS.MC", "BBVA.MC", "SAB.MC", "SAN.MC", "BKT.MC", "CABK.MC", "CLNX.MC", "ANE.MC", "ENG.MC", "ELE.MC", "FER.MC", "FDR.MC", "GRF.MC", "IAG.MC", "IBE.MC", "ITX.MC", "IDR.MC", "COL.MC", "ROVI.MC", "LOG.MC", "MAP.MC", "MEL.MC", "MRL.MC", "NTGY.MC", "RED.MC", "REP.MC", "SCYR.MC", "SLR.MC", "TEF.MC", "UNI.MC"]

end_date = datetime.now(tz=pytz.timezone('Europe/Madrid'))
start_date = end_date - timedelta(days=60)

def get_stock_data(stock):
    try:
        ticker = yf.Ticker(stock)
        data = ticker.history(start=start_date, end=end_date)
        data["company"] = ticker.info["longName"]
        return data
    except:
        print(f"\033[91mError retrieving data for {stock}\033[0m")
        return None

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_stock_data, stock) for stock in ibex35]
    results = wait(futures)

data = [f.result() for f in results.done if f.result() is not None]
df1 = pd.concat(data)
df1.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits,company
Date,,,,,,,,
2023-03-13 00:00:00+01:00,22.700001,22.700001,22.059999,22.100000,277759,0.0,0.0,Compañía de Distribución Integral Logista Hold...
2023-03-14 00:00:00+01:00,22.080000,22.340000,21.940001,22.299999,278874,0.0,0.0,Compañía de Distribución Integral Logista Hold...
2023-03-15 00:00:00+01:00,22.299999,22.639999,22.000000,22.040001,360596,0.0,0.0,Compañía de Distribución Integral Logista Hold...
2023-03-16 00:00:00+01:00,22.200001,22.420000,22.020000,22.299999,335218,0.0,0.0,Compañía de Distribución Integral Logista Hold...
2023-03-17 00:00:00+01:00,22.400000,22.540001,22.160000,22.280001,585118,0.0,0.0,Compañía de Distribución Integral Logista Hold...


In [ ]:
df1 = SMA(df1, periodo = 10)
df1 = EMA(df1, periodo = 10)
df1 = MACD(df1, window_slow = 6, window_fast = 12, window_sign = 9)
df1 = RSI(df1, periodo = 14)
df1 = BOLLINGER(df1, periodo = 21, dev = 2)
df1 = STD_DEV(df1, periodo = 14)
df1 = CCI(df1, periodo = 14, constant = 0.015)
df1 = ESTOCASTICO(df1, periodo = 14, smooth = 3)
df1 = WILLIAMSR(df1, periodo = 14)
df1 = ADI(df1)
df1 = CMF(df1, periodo = 21)
df1 = PSAR(df1, step = 0.02, maxstep = 0.2)
df1 = ATR(df1, periodo = 14) 
df1 = ADX(df1, periodo = 14)
df1 = ROC(df1, periodo = 12)
df1 = OBV(df1)
df1 = MFI(df1, periodo = 14)
df1 = ICHIMOKU (df1, p_bajo = 9, p_medio = 26, p_alto = 52)
df1 = DPO(df1, periodo = 20)
df1 = NVI(df1)

In [ ]:
# Crea las nuevas variables basadas en indicadores técnicos bursátiles

df1 = agregar_pre_open(df1)
df1 = agregar_gaps(df1)
df1 = date_features(df1)
df1.reset_index(drop=True, inplace=True)
df1 = days_to_closure_and_profit(df1)


In [ ]:
#movemos los indices
df1 = shift_rows(df1)

In [ ]:
# Pasar 'CLOSURE' a la primera posición en la tabla porque es nuestra label 
cols = df1.columns.tolist()                        # Obtener los nombres de las columnas como una lista
cols.insert(0, cols.pop(cols.index('CLOSURE')))   # Eliminar la columna CLOSURE de su posición original y agregarla en la posición 0
df1 = df1.reindex(columns=cols)                     # Reorganizar el DataFrame según la nueva lista de columnas

**Y como solo queremos modelar los gaps negativos, eliminamos todas las filas que tengan gaps iguales o mayores que 0**

In [ ]:
# Selecciona todas las filas en las que el valor de "GAP_€" es negativo
df1 = df1.loc[df1['GAP_€'] < 0]

In [ ]:
df1.to_csv("ibex_para_predecir.csv", index=False)
files.download("ibex_para_predecir.csv")
print("Descarga completa.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Descarga completa.
